In [67]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [68]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from forcateri.data.dataprovider import DataProvider, SeriesRole
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
from forcateri.data.timeseries import TimeSeries

In [69]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
# baltbest = BaltBestAggregatedAPIData(
#     name='test', 
#     url="baltbest_url", 
#     local_copy="/home/user/DFKI/forcateri/_data/showcase_data.csv",
#     target = 'q_hca',
#     group_col = 'room_id',
#     time_col = 'datetime',
#     known = 'temperature_outdoor_avg',
#     observed = ['temperature_1_max', 'temperature_2_max','temperature_room_avg'],)
baltbest = BaltBestAggregatedAPIData(
    name='test', 
    url="baltbest_url", 
    local_copy="/home/user/DFKI/forcateri/_data/hourly_data.csv",
    target = 'delta',
    group_col = 'room_id',
    time_col = 'rounded_ts',
    known = 'outside_temp',
    observed = ['max_temperature_1', 'max_temperature_2','room_temperature'],)

In [71]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [72]:

#baltbest.get_data()


In [73]:
# roles = {
#     'q_hca': SeriesRole.TARGET, 
#     'temperature_outdoor_avg':SeriesRole.KNOWN, 
#     'temperature_1_max':SeriesRole.OBSERVED, 
#     'temperature_2_max':SeriesRole.OBSERVED,
#     'temperature_room_avg':SeriesRole.OBSERVED,}
#['temperature_1_max', 'temperature_2_max','temperature_room_avg']
roles = {
    'delta': SeriesRole.TARGET, 
    'outside_temp':SeriesRole.KNOWN, 
    'max_temperature_1':SeriesRole.OBSERVED, 
    'max_temperature_1':SeriesRole.OBSERVED,
    'room_temperature':SeriesRole.OBSERVED,}

In [74]:
start = pd.Timestamp(2021, 1, 3, 4,tz=0)
end = pd.Timestamp(2021, 1, 8, 4,tz=0)
dataprovider = DataProvider(data_sources=[baltbest], roles=roles, splits=(100,200))

2025-05-26 14:51:43,630 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,643 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,656 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,668 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,681 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,695 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,707 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,720 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,733 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,746 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,758 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:43,771 - INFO -

In [75]:
train = dataprovider.get_train_set()
val = dataprovider.get_val_set()
test = dataprovider.get_test_set()

2025-05-26 14:51:45,008 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,010 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,013 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,015 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,017 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,019 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,022 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,024 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,026 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,029 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,031 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-05-26 14:51:45,033 - INFO -

In [76]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
quantiles = [0.1, 0.5, 0.9]
dartstcn = DartsTCNModel( quantiles=quantiles)

In [77]:
dartstcn.fit(train_data = train, val_data=val)

/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodel

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [78]:
num_forecast_steps = 2
#num_monte_carlo_samples = 50 # A sufficient number of samples to approximate the quantiles

#Here prediction is darts timeseries
prediction = dartstcn.predict(
    data=val, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    n=num_forecast_steps,
)

/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/user/DFKI/forcateri/forcateri/model/dartsmodels/dartsmodel

Predicting: |          | 0/? [00:00<?, ?it/s]

In [79]:
from IPython.display import display, Markdown
def mprint(s): display(Markdown(s))

In [80]:
prediction_ts = dartstcn.to_time_series(prediction)

2025-05-26 14:52:06,495 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,498 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,502 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,506 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,509 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,513 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,516 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,521 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,526 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,530 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26 14:52:06,533 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-05-26

In [81]:
prediction_ts[0].data

feature                                target                    
representation                            0.1       0.5       0.9
offset          time_stamp                                       
0 days 01:00:00 2019-06-09 09:00:00  0.002359 -0.003704  0.013214
0 days 02:00:00 2019-06-09 10:00:00  0.001875 -0.003660  0.012803

In [82]:
prediction_df = prediction[0].to_dataframe()
prediction_ts = TimeSeries(prediction_df,representation="quantile", quantiles=[0.1,0.5,0.9])
prediction_ts.data

2025-05-26 14:52:06,979 - INFO - TimeSeries initialized from compatible-format DataFrame.


feature                       target                    
representation                   0.1       0.5       0.9
offset time_stamp                                       
0 days 2019-06-09 09:00:00  0.002359 -0.003704  0.013214
       2019-06-09 10:00:00  0.001875 -0.003660  0.012803